<a href="https://colab.research.google.com/github/Aasthapaudel/-Imagine-cup-cloud-skill-Challenge-/blob/main/chatbotnepali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nagargpt .

In [3]:
!pip install transformers datasets bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch


In [5]:
model_name = "unsloth/llama-3-8b-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
dataset = load_dataset("teksingh/NagarGPT-dataset0")  # replace with your dataset path if local


In [ ]:
def preprocess_function(examples):
    inputs = [f"Question: {q}\nAnswer:" for q in examples['मानव']]
    answers = examples['सहायक']
    model_inputs = tokenizer(
        inputs,
        text_target=answers,
        max_length=512,
        truncation=True,
        padding="max_length"  # Ensures all sequences have the same length
    )
    return model_inputs


In [ ]:
training_args = TrainingArguments(
    output_dir="./NepaliGPT-finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=1000,
    logging_dir="./logs",
    fp16=True  # enable mixed precision if GPU supports it
)


In [ ]:
print(dataset)


In [ ]:
# Split the dataset into 90% training and 10% validation
dataset = dataset["train"].train_test_split(test_size=0.1)
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Now you can access both "train" and "validation" splits
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]


In [ ]:
!pip install peft


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

# Load tokenizer and quantized model
model_name = "unsloth/llama-3-8b-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,   # Rank of LoRA matrices, adjustable based on available resources
    lora_alpha=32,
    lora_dropout=0.1
)

# Add LoRA adapters to the model
model = get_peft_model(model, lora_config)


In [ ]:
from transformers import DataCollatorForSeq2Seq

# Initialize the data collator with padding for consistent batch lengths
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", None),  # Only if validation split exists
    data_collator=data_collator  # Ensures proper padding
)


In [24]:
trainer.train_dataset()


ValueError: Expected input batch_size (164) to match target batch_size (1228).

In [23]:
trainer.save_model("NepaliGPT-finetuned")


In [26]:
input_text = "जन्मदर्ता गर्दा आवश्यक कागजपत्र के के चाहिन्छ?"  # replace with a sample question in Nepali
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


जन्मदर्ता गर्दा आवश्यक कागजपत्र के के चाहिन्छ? (What documents are required to get birth certificate in Nepal?)
जन्मदर्ता गर्दा आवश्यक कागजपत्र के चाहिन्छ?
जन्मदर्ता गर्दा आवश्यक कागजपत्र क


In [1]:
def preprocess_function(examples):
    inputs = [f"प्रश्न: {q}\nउत्तर:" for q in examples['मानव']]
    answers = examples['सहायक']
    model_inputs = tokenizer(inputs, text_target=answers, max_length=512, truncation=True, padding="max_length")
    return model_inputs


In [2]:
input_text = "जन्मदर्ता गर्दा आवश्यक कागजपत्र के के चाहिन्छ?"  # replace with a sample question in Nepali
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


NameError: name 'tokenizer' is not defined